In [ ]:
import os
import json

# Stock class representing a stock with price and shares
class Stock:
    def __init__(self, price_per_share, shares_owned=0):
        self.price_per_share = price_per_share
        self.shares_owned = shares_owned

    def buy_shares(self, quantity):
        """Add shares to the stock."""
        self.shares_owned += quantity

    def sell_shares(self, quantity):
        """Sell shares if there are enough owned shares."""
        if quantity > self.shares_owned:
            print("Error: Not enough shares to sell.")
            return False
        self.shares_owned -= quantity
        return True

    def to_dict(self):
        """Convert stock information to a dictionary for saving."""
        return {
            'price_per_share': self.price_per_share,
            'shares_owned': self.shares_owned
        }

    @classmethod
    def from_dict(cls, data):
        """Create a stock object from a dictionary."""
        return cls(data['price_per_share'], data['shares_owned'])


# Portfolio class to represent a collection of stocks
class Portfolio:
    def __init__(self, name):
        self.name = name
        self.stocks = {}

    def add_stock(self, price_per_share, quantity):
        """Add a stock to the portfolio or increase quantity if it already exists."""
        if price_per_share in self.stocks:
            self.stocks[price_per_share].buy_shares(quantity)
        else:
            self.stocks[price_per_share] = Stock(price_per_share, quantity)

    def sell_stock(self, price_per_share, quantity):
        """Sell a stock from the portfolio if there are enough shares."""
        if price_per_share in self.stocks:
            return self.stocks[price_per_share].sell_shares(quantity)
        else:
            print(f"Error: No stock found with price R{price_per_share:.2f}.")
            return False

    def display_portfolio(self):
        """Display all stocks in the portfolio."""
        print(f"Portfolio: {self.name}")
        for stock in self.stocks.values():
            print(f"  Stock @ R{stock.price_per_share:.2f}: {stock.shares_owned} shares")

    def to_dict(self):
        """Convert portfolio to a dictionary for saving."""
        return {
            'name': self.name,
            'stocks': {price: stock.to_dict() for price, stock in self.stocks.items()}
        }

    @classmethod
    def from_dict(cls, data):
        """Create a portfolio object from a dictionary."""
        portfolio = cls(data['name'])
        for price, stock_data in data['stocks'].items():
            portfolio.stocks[float(price)] = Stock.from_dict(stock_data)
        return portfolio


# multiple portfolios and balances
class StockPortfolioManager:
    def __init__(self):
        self.portfolios = {}
        self.balance = ()
        self.load_data()

    def add_portfolio(self, name):
        """Add a new portfolio to the manager."""
        if name in self.portfolios:
            print(f"Error: Portfolio '{name}' already exists.")
            return False
        self.portfolios[name] = Portfolio(name)
        return True

    def deposit_money(self, amount):
        """Deposit money into the balance."""
        if amount <= 0:
            print("Error: Deposit amount must be greater than zero.")
            return False
        self.balance += amount
        return True

    def withdraw_money(self, amount):
        """Withdraw money from the balance."""
        if amount <= 0:
            print("Error: Withdrawal amount must be greater than zero.")
            return False
        if amount > self.balance:
            print("Error: Insufficient funds to withdraw.")
            return False
        self.balance -= amount
        return True

    def buy_stock(self, portfolio_name, price_per_share, quantity):
        """Buy stock and add it to the portfolio if funds are available."""
        if portfolio_name not in self.portfolios:
            print(f"Error: Portfolio '{portfolio_name}' does not exist.")
            return False
        cost = price_per_share * quantity
        if cost > self.balance:
            print("Error: Insufficient funds to buy the stock.")
            return False
        self.balance -= cost
        self.portfolios[portfolio_name].add_stock(price_per_share, quantity)
        return True

    def sell_stock(self, portfolio_name, price_per_share, quantity):
        """Sell stock from the portfolio if possible."""
        if portfolio_name not in self.portfolios:
            print(f"Error: Portfolio '{portfolio_name}' does not exist.")
            return False
        return self.portfolios[portfolio_name].sell_stock(price_per_share, quantity)

    def display_balance(self):
        """Display the current balance."""
        print(f"Current balance: R{self.balance:.2f}")

    def display_all_portfolios(self):
        """Display all portfolios and their stocks."""
        if not self.portfolios:
            print("No portfolios available.")
        for portfolio in self.portfolios.values():
            portfolio.display_portfolio()

    def display_specific_portfolio(self, name):
        """Display a specific portfolio by name."""
        if name in self.portfolios:
            self.portfolios[name].display_portfolio()
        else:
            print(f"Error: No portfolio found with the name '{name}'.")

    def save_data(self):
        """Save portfolio data to a file."""
        data = {
            'balance': self.balance,
            'portfolios': {name: portfolio.to_dict() for name, portfolio in self.portfolios.items()}
        }
        with open('portfolio_data.json', 'w') as file:
            json.dump(data, file)

    def load_data(self):
        """Load portfolio data from a file."""
        if os.path.exists('portfolio_data.json'):
            with open('portfolio_data.json', 'r') as file:
                data = json.load(file)
                self.balance = data.get('balance', 10000.00)
                self.portfolios = {
                    name: Portfolio.from_dict(portfolio_data)
                    for name, portfolio_data in data.get('portfolios', {}).items()
                }


# main menu 
def display_menu():
    print("""
Stock Portfolio Manager Menu:
1. Add a new portfolio
2. Add stock to portfolio
3. Buy stock
4. Sell stock
5. Deposit money
6. Withdraw money
7. Display balance
8. Display all portfolios
9. Display specific portfolio
0. Exit
""")


# Prompt function 
def main():
    manager = StockPortfolioManager()

    while True:
        display_menu()
        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter portfolio name: ")
            if manager.add_portfolio(name):
                print(f"Portfolio '{name}' added successfully.")

        elif choice == "2":
            portfolio_name = input("Enter portfolio name: ")
            price = float(input("Enter price per share (R): "))
            quantity = int(input("Enter number of shares: "))
            if manager.buy_stock(portfolio_name, price, quantity):
                print(f"Added {quantity} shares @ R{price:.2f} to {portfolio_name}.")

        elif choice == "3":
            portfolio_name = input("Enter portfolio name: ")
            price = float(input("Enter price per share (R): "))
            quantity = int(input("Enter number of shares to buy: "))
            if manager.buy_stock(portfolio_name, price, quantity):
                print(f"Bought {quantity} shares @ R{price:.2f}.")

        elif choice == "4":
            portfolio_name = input("Enter portfolio name: ")
            price = float(input("Enter stock price (R): "))
            quantity = int(input("Enter number of shares to sell: "))
            if manager.sell_stock(portfolio_name, price, quantity):
                print(f"Sold {quantity} shares @ R{price:.2f}.")

        elif choice == "5":
            amount = float(input("Enter amount to deposit (R): "))
            if manager.deposit_money(amount):
                print(f"Deposited R{amount:.2f}.")

        elif choice == "6":
            amount = float(input("Enter amount to withdraw (R): "))
            if manager.withdraw_money(amount):
                print(f"Withdrew R{amount:.2f}.")

        elif choice == "7":
            os.system('clear')
            manager.display_balance()

        elif choice == "8":
            os.system('clear')
            manager.display_all_portfolios()

        elif choice == "9":
            name = input("Enter portfolio name: ")
            manager.display_specific_portfolio(name)

        elif choice == "0":
            manager.save_data()
            break

        else:
            print("Invalid choice, try again.")

if __name__ == "__main__":
    main()



Stock Portfolio Manager Menu:
1. Add a new portfolio
2. Add stock to portfolio
3. Buy stock
4. Sell stock
5. Deposit money
6. Withdraw money
7. Display balance
8. Display all portfolios
9. Display specific portfolio
0. Exit

